In [18]:
import openai
import os

In [2]:
from pydantic import BaseModel, ValidationError
from typing import List

class MeetingNotes(BaseModel):
    summary: str
    action_items: List[str]


In [12]:
import json
def safe_parse_json(raw_text: str) -> dict:
    try:
        start = raw_text.index("{")
        end = raw_text.rindex("}") + 1
        return json.loads(raw_text[start:end])
    except Exception as e:
        raise ValueError(f"Could not extract JSON: {e}")


In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
llm = AzureChatOpenAI(
    deployment_name="",
    temperature=0.2,
    openai_api_key="",
    openai_api_base="",
    openai_api_version="2024-12-01-preview"
)

def build_prompt(transcript):
    return f"""You are a meeting assistant.
1. Summarize the meeting transcript below in exactly two sentences.
2. Then list all action items mentioned, each as a separate bullet beginning with a dash.
Return the result strictly as JSON with keys "summary" and "action_items".

Transcript:
{transcript}
"""

def extract_meeting_notes(transcript: str) -> dict:
    try:
        prompt = build_prompt(transcript)
        response = llm([HumanMessage(content=prompt)])
        raw_output = response.content
        parsed_dict = safe_parse_json(raw_output)
        return MeetingNotes.model_validate(parsed_dict).dict()

    except Exception:
        retry_msg = [
            SystemMessage(content="Please output valid JSON only."),
            HumanMessage(content=prompt)
        ]
        response = llm(retry_msg)
        raw_output = response.content
        parsed_dict = safe_parse_json(raw_output)
        return MeetingNotes.model_validate(parsed_dict).dict()


c:\One-Drive\OneDrive - Tredence\Desktop\databricks_genai_hackt-main\agenticAI\AgenticAIWorkspace\.venv\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://eastus.api.cognitive.microsoft.com/ to https://eastus.api.cognitive.microsoft.com/openai.
  warnings.warn(
c:\One-Drive\OneDrive - Tredence\Desktop\databricks_genai_hackt-main\agenticAI\AgenticAIWorkspace\.venv\Lib\site-packages\langchain_community\chat_models\azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\One-Drive\OneDrive - Tredence\Desktop\databricks_genai_hackt-main\agenticAI\AgenticAIWorkspace\.v

In [19]:
test_cases = [
    {
        "name": "Sample Input 1",
        "transcript": """Host: Let’s kick off our marketing sync.
Emma: The social campaign draft is 80% done; I’ll share it today.
Frank: I spoke with the design team—they’ll deliver assets by Tuesday.
Emma: Once we have assets, I’ll schedule the ads for next week.
George: Reminder: submit your budget requests before end of day.
Host: Noted. I’ll send out the final budget spreadsheet."""
    },
    {
        "name": "Sample Input 2",
        "transcript": """Alice: Welcome everyone. Today we need to finalize the Q3 roadmap.
Bob: I’ve emailed the updated feature list—please review by Friday.
Carol: I’ll set up the user‐testing sessions next week.
Dan: Let’s push the new UI mockups to staging on Wednesday.
Alice: Great. Also, can someone compile the stakeholder feedback into a slide deck?
Bob: I can handle the slide deck by Monday.
Alice: Thanks, team. Meeting adjourned."""
    },
    {
        "name": "Sample Input 3",
        "transcript": """Priya: We’ve identified a bottleneck in the checkout API.
Carlos: I’ll benchmark the current response times by tomorrow.
Priya: After that, optimize the database indices.
Carlos: Sure, I’ll propose index changes by Thursday.
Sara: Also, draft a rollback plan in case of production issues.
Carlos: I’ll include that in my ticket."""
    },
    {
        "name": "Sample Input 4",
        "transcript": """HR: We need to update the travel‐expense policy.
Alex: I’ll draft the new policy doc and share with legal.
HR: Please include guidelines on per diem limits.
Legal: I’ll review and provide feedback within three days.
HR: Once finalized, schedule a companywide announcement."""
    },
    {
        "name": "Sample Input 5",
        "transcript": """Jordan: The app's user onboarding is still confusing to new users.
Maya: I’ll draft a revised onboarding flow by Friday.
Jordan: Can we also A/B test it with two different copy styles?
Maya: Yes, I’ll coordinate with UX.
Liam: I’ll set up the tracking in Mixpanel once we finalize the version."""
    }
]
for case in test_cases:
    print(f"\n {case['name']}")
    try:
        output = extract_meeting_notes(case["transcript"])
        print("Summary:", output["summary"])
        print("Action Items:")
        for item in output["action_items"]:
            print(item)
    except Exception as e:
        print("Failed to process:", str(e))



 Sample Input 1


C:\Users\vyshagh.a\AppData\Local\Temp\ipykernel_8576\3232947862.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return MeetingNotes.model_validate(parsed_dict).dict()


Summary: The marketing sync focused on the progress of the social campaign and the timeline for asset delivery. Action items include sharing the campaign draft, scheduling ads, submitting budget requests, and sending out the final budget spreadsheet.
Action Items:
- Emma will share the social campaign draft today.
- Frank will ensure the design team delivers assets by Tuesday.
- Emma will schedule the ads for next week once assets are received.
- Team members need to submit their budget requests before the end of the day.
- The host will send out the final budget spreadsheet.

 Sample Input 2


C:\Users\vyshagh.a\AppData\Local\Temp\ipykernel_8576\3232947862.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return MeetingNotes.model_validate(parsed_dict).dict()


Summary: The team met to finalize the Q3 roadmap, discussing various tasks and deadlines. Bob will compile the stakeholder feedback into a slide deck by Monday, while Carol will set up user-testing sessions next week and Dan will push the new UI mockups to staging on Wednesday.
Action Items:
- Review the updated feature list by Friday.
- Set up user-testing sessions next week.
- Push the new UI mockups to staging on Wednesday.
- Compile the stakeholder feedback into a slide deck by Monday.

 Sample Input 3


C:\Users\vyshagh.a\AppData\Local\Temp\ipykernel_8576\3232947862.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return MeetingNotes.model_validate(parsed_dict).dict()


Summary: The team identified a bottleneck in the checkout API and discussed steps to address it. Carlos will benchmark response times, propose index changes, and draft a rollback plan by the specified deadlines.
Action Items:
- Carlos will benchmark the current response times by tomorrow.
- Carlos will propose index changes by Thursday.
- Carlos will draft a rollback plan in case of production issues.

 Sample Input 4


C:\Users\vyshagh.a\AppData\Local\Temp\ipykernel_8576\3232947862.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return MeetingNotes.model_validate(parsed_dict).dict()


Summary: The team discussed the need to update the travel-expense policy, with Alex agreeing to draft a new document. Legal will review the draft and provide feedback, after which a companywide announcement will be scheduled once finalized.
Action Items:
- Alex will draft the new travel-expense policy document.
- Legal will review the draft and provide feedback within three days.
- HR will schedule a companywide announcement once the policy is finalized.

 Sample Input 5
Summary: The team discussed the confusion new users face during the app's onboarding process and agreed to revise the onboarding flow. Maya will draft the new flow by Friday, and Liam will set up tracking in Mixpanel after finalization.
Action Items:
- Maya will draft a revised onboarding flow by Friday.
- Maya will coordinate with UX for A/B testing.
- Liam will set up the tracking in Mixpanel once the version is finalized.


C:\Users\vyshagh.a\AppData\Local\Temp\ipykernel_8576\3232947862.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return MeetingNotes.model_validate(parsed_dict).dict()
